<a href="https://colab.research.google.com/github/chaufe/testcases/blob/main/colab/OpenRoad__invalid_pin_access_check_on_pads.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Testcase to show OpenRoad to error-out on pin acces check on pad opening**

### Environment and Software Setup
- setup environment
- install conda-eda, GF180MCU PDK and EDA software
- install and patch helper software

In [1]:
#@title Install a few packages required later-on { display-mode: "form" }
!dpkg --configure -a
!apt install python3-click

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  python3-colorama
The following NEW packages will be installed:
  python3-click python3-colorama
0 upgraded, 2 newly installed, 0 to remove and 13 not upgraded.
Need to get 88.7 kB of archives.
After this operation, 395 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 python3-colorama all 0.4.3-1build1 [23.9 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 python3-click all 7.0-3 [64.8 kB]
Fetched 88.7 kB in 1s (95.6 kB/s)
Selecting previously unselected package python3-colorama.
(Reading database ... 123069 files and directories currently installed.)
Preparing to unpack .../python3-colorama_0.4.3-1build1_all.deb ...
Unpacking python3-colorama (0.4.3-1build1) ...
Selecting previously unselected package python3-click.
Preparing to unpack .../python3-click_7.0-3_all.deb ...
Unpacki

In [2]:
#@title Install Anaconda environment {display-mode: "form"}
CONDA_PREFIX = '/content/eda'
%env CONDA_PREFIX = {CONDA_PREFIX}
!echo 'Will install CONDA EDA to {CONDA_PREFIX}'
!test -e Install.sh || wget -O Install.sh https://repo.anaconda.com/archive/Anaconda3-2023.03-1-Linux-x86_64.sh
!test -d {CONDA_PREFIX} || bash Install.sh -b -p {CONDA_PREFIX}

env: CONDA_PREFIX=/content/eda
Will install CONDA EDA to /content/eda
--2023-06-28 13:42:12--  https://repo.anaconda.com/archive/Anaconda3-2023.03-1-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 902411137 (861M) [application/x-sh]
Saving to: ‘Install.sh’

Install.sh          100%[===================>] 860.61M   108MB/s    in 9.8s    

2023-06-28 13:42:22 (87.8 MB/s) - ‘Install.sh’ saved [902411137/902411137]

PREFIX=/content/eda
Unpacking payload ...
                                                                                       
Installing base environment...





Preparing transaction: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ 

In [3]:
#@title Update conda packages {display-mode: "form"}
!{CONDA_PREFIX}/bin/conda update --yes --all

Solving environment: | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done

## Package Plan ##

  environment location: /content/eda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    anaconda-client-1.11.3     |  py310h06a4308_0         142 KB
    anaconda-navigator-2.4.2   |  py310h06a4308_0         5.4 MB
    beautifulsoup4-4.12.2      |  py310h06a4308_0         212 KB
    ca-certificates-2023.05.30 |       h06a4308_0         120 KB
    certifi-2023.5.7           |  py310h06a4308_0         152 KB
    conda-23.5.0               |  py310h06a4308_0         1.0 MB
    conda-build-3.25.0         |

In [4]:
#@title Install EDA software { display-mode: "form" }

# required extra packages
!{CONDA_PREFIX}/bin/conda install --yes libboost fmt==8.1.1 spdlog
!{CONDA_PREFIX}/bin/conda install --yes litex-hub::tcllib

# use hard-coded packages as version resolution does not work as expected
for pkg in [
    'https://anaconda.org/LiteX-Hub/klayout/0.28.9_2_2_g508b52374/download/linux-64/klayout-0.28.9_2_2_g508b52374-20230606_125334_py37.tar.bz2',
    'https://anaconda.org/LiteX-Hub/magic/8.3.404_0_g482d753/download/linux-64/magic-8.3.404_0_g482d753-20230606_125334.tar.bz2',
    'https://anaconda.org/LiteX-Hub/netgen/1.5.254_0_g02d7a1b/download/linux-64/netgen-1.5.254_0_g02d7a1b-20230606_125334.tar.bz2',
    'https://anaconda.org/LiteX-Hub/open_pdks.gf180mcuc/1.0.419_0_g3df14f8/download/noarch/open_pdks.gf180mcuc-1.0.419_0_g3df14f8-20230606_125334.tar.bz2',
    'https://anaconda.org/LiteX-Hub/openlane/2023.04.12_0_g23e367a9/download/noarch/openlane-2023.04.12_0_g23e367a9-20230412_103222.tar.bz2',
    'https://anaconda.org/LiteX-Hub/openroad/2.0_8518_g8901000e3/download/linux-64/openroad-2.0_8518_g8901000e3-20230606_125334.tar.bz2',
    'https://anaconda.org/LiteX-Hub/yosys/0.30_17_g8b2a00102/download/linux-64/yosys-0.30_17_g8b2a00102-20230606_125334_py37.tar.bz2',
    'https://repo.anaconda.com/pkgs/main/linux-64/ruby-3.1.4-h359c5ee_0.conda',
    'https://repo.anaconda.com/pkgs/main/linux-64/python-3.7.16-h7a1cb2a_0.conda',
]:
  print("Installing " + pkg)
  !{CONDA_PREFIX}/bin/conda install --yes {pkg}

Solving environment: \ unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done

## Package Plan ##

  environment location: /content/eda

  added / updated specs:
    - fmt==8.1.1
    - libboost
    - spdlog


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    fmt-8.1.1                  |       hd09550d_1         174 KB
    libboost-1.73.0            |      h28710b8_12        13.8 MB
    spdlog-1.9.2               |       hd09550d_0         199 KB
    ------------------------------------------------------------
                                           Total:        14.2 MB

The following NEW packages will be INSTALLED:

  fmt                pkgs/main/linux-64::fmt-8.1.1-hd09550d_1 
  libboost 

In [5]:
#@title Setup remaining variables { display-mode: 'form'}
%env CONDA_PREFIX = {CONDA_PREFIX}
import os
%env PATH = {CONDA_PREFIX + '/bin:' + os.getenv('PATH')}
%env PDK_ROOT = {CONDA_PREFIX + '/share/pdk'}
%env PDK = gf180mcuC
%env IO_LIB_PATH = {CONDA_PREFIX + '/share/pdk/gf180mcuC/libs.ref/gf180mcu_fd_io'}
%env SRAM_LIB_PATH = {CONDA_PREFIX + '/share/pdk/gf180mcuC/libs.ref/gf180mcu_fd_ip_sram'}

env: CONDA_PREFIX=/content/eda
env: PATH=/content/eda/bin:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
env: PDK_ROOT=/content/eda/share/pdk
env: PDK=gf180mcuC
env: IO_LIB_PATH=/content/eda/share/pdk/gf180mcuC/libs.ref/gf180mcu_fd_io
env: SRAM_LIB_PATH=/content/eda/share/pdk/gf180mcuC/libs.ref/gf180mcu_fd_ip_sram


In [6]:
#@title Fix PYTHONPATH to make OpenROAD happy
%env PYTHONPATH = /env/python:/content/eda/lib/python3.10:/content/eda/lib/python310.zip:/content/eda/lib/python3.10/lib-dynload:/content/eda/lib/python3.10/site-packages
import sys
for path in sys.path:
  print(path)

env: PYTHONPATH=/env/python:/content/eda/lib/python3.10:/content/eda/lib/python310.zip:/content/eda/lib/python3.10/lib-dynload:/content/eda/lib/python3.10/site-packages
/content
/env/python
/usr/lib/python310.zip
/usr/lib/python3.10
/usr/lib/python3.10/lib-dynload

/usr/local/lib/python3.10/dist-packages
/usr/lib/python3/dist-packages
/usr/local/lib/python3.10/dist-packages/IPython/extensions
/root/.ipython


In [24]:
#@title Patch GF180MCU IP { display-mode: "form" }
!wget https://raw.githubusercontent.com/chaufe/testcases/main/patches/gf180mcu_fd_io.lef.patch
!patch -p0 < gf180mcu_fd_io.lef.patch

--2023-06-28 14:42:43--  https://raw.githubusercontent.com/chaufe/testcases/main/patches/gf180mcu_fd_io.lef.patch
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1521 (1.5K) [text/plain]
Saving to: ‘gf180mcu_fd_io.lef.patch’

gf180mcu_fd_io.lef. 100%[===================>]   1.49K  --.-KB/s    in 0s      

2023-06-28 14:42:43 (13.4 MB/s) - ‘gf180mcu_fd_io.lef.patch’ saved [1521/1521]

patching file eda/share/pdk/gf180mcuC/libs.ref/gf180mcu_fd_io/lef/gf180mcu_fd_io.lef
Hunk #2 succeeded at 832 (offset 3 lines).
Hunk #3 succeeded at 1695 (offset 12 lines).
Hunk #4 succeeded at 3973 (offset 152 lines).
Hunk #5 succeeded at 4766 (offset 152 lines).
Hunk #6 succeeded at 7775 (offset 152 lines).
Hunk #7 succeeded at 8586 (offset 156 lines).


In [10]:
#@title Write testcase RTL { display-mode: "form" }
%%writefile "testcase.v"
module testcase (input clk,
                 input din,
                 output wire dout);

   wire clk_int;
   wire din_int;
   reg  dout_int;


   // pads
   (* keep *) gf180mcu_fd_io__bi_t pad_dout(
                                            .PU(1'b0), .PD(1'b0),
                                            .OE(1'b1), .IE(1'b0),
                                            .PDRV0(1'b0), .PDRV1(1'b0),
                                            .CS(1'b0), .SL(1'b0),
                                            .PAD(dout),
                                            .A(dout_int),
                                            .Y()
                                            );

   (* keep *) gf180mcu_fd_io__in_s pad_clk(
                                           .PU(1'b0), .PD(1'b0),
                                           .PAD(clk),
                                           .Y(clk_int)
                                           );

   (* keep *) gf180mcu_fd_io__in_s pad_din(
                                           .PU(1'b0), .PD(1'b0),
                                           .PAD(din),
                                           .Y(din_int)
                                           );

    always @(posedge clk_int)
      dout_int <= din_int;

endmodule


Writing testcase.v


In [11]:
#@title Write config.tcl to control OpenLANE { display-mode: "form" }
%%writefile "config.tcl"

set ::env(DESIGN_NAME) "testcase"

set ::env(VERILOG_FILES) [list "testcase.v" ]

set ::env(EXTRA_LIBS) [list \
  $::env(IO_LIB_PATH)/liberty/gf180mcu_fd_io__tt_025C_5v00.lib \
  $::env(IO_LIB_PATH)/liberty/gf180mcu_fd_io__ff_n40C_5v50.lib \
  $::env(IO_LIB_PATH)/liberty/gf180mcu_fd_io__ff_125C_5v50.lib \
  $::env(IO_LIB_PATH)/liberty/gf180mcu_fd_io__ss_125C_4v50.lib \
]

set ::env(EXTRA_LEFS) [list \
]

#set ::env(BASE_SDC_FILE) "testcase.sdc"

set ::env(EXTRA_GDS_FILES) [list \
  $::env(IO_LIB_PATH)/gds/gf180mcu_fd_io.gds \
]

set ::env(DESIGN_IS_CORE) 1
set ::env(FP_PDN_CORE_RING) 1

set ::env(GRT_ALLOW_CONGESTION) 1
set ::env(DRT_OPT_ITERS) 10

set ::env(DIODE_INSERTION_STRATEGY) 0

set ::env(USE_GPIO_PADS) 1
set ::env(GPIO_PADS_LEF) $::env(IO_LIB_PATH)/lef/gf180mcu_fd_io.lef
unset ::env(GPIO_PADS_VERILOG)
set ::env(FP_IO_MODE) 1

set ::env(FP_PDN_UPPER_LAYER) Metal5
set ::env(FP_PDN_LOWER_LAYER) Metal4
set ::env(FP_PDN_RAILS_LAYER) Metal1
set ::env(FP_PDN_CORE_RING_VWIDTH) 5.0
set ::env(FP_PDN_CORE_RING_HWIDTH) 5.0
set ::env(FP_PDN_CORE_RING_VSPACING) 1.0
set ::env(FP_PDN_CORE_RING_HSPACING) 1.0

set ::env(SYNTH_NO_FLAT) 0
set ::env(SYNTH_SHARE_RESOURCES) 1

set ::env(CLOCK_TREE_SYNTH) 1
set ::env(CLOCK_PORT) "clk"
set ::env(CLOCK_PERIOD) 100

set ::env(FP_SIZING) absolute
set ::env(DIE_AREA) "0 0 1460 1460"
set ::env(CORE_AREA) "370 370 1090 1090"

set ::env(PL_TARGET_DENSITY) 0.65

# MAGIC GDS-out controls
## ... try GDS with hierarchy
set ::env(MAGIC_DISABLE_HIER_GDS) 0
set ::env(MAGIC_GENERATE_LEF) 0
set ::env(MAGIC_GENERATE_MAGLEF) 0

Writing config.tcl


In [29]:
#@title Use updated ioplace script to create pad ring { display-mode: "form" }
%%writefile "eda/share/openlane/scripts/openroad/ioplacer.tcl"
# Copyright 2020-2022 Efabless Corporation
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
source $::env(SCRIPTS_DIR)/openroad/common/io.tcl
read

if { [info exists ::env(CONTEXTUAL_IO_FLAG)] } {
	read_lef $::env(placement_tmpfiles)/top_level.lef
}

if {$::env(FP_IO_HLENGTH) != "" && $::env(FP_IO_HLENGTH) != ""} {
	set_pin_length -hor_length $::env(FP_IO_HLENGTH) \
		-ver_length $::env(FP_IO_VLENGTH)
}

if {$::env(FP_IO_HEXTEND) != "0" && $::env(FP_IO_VEXTEND) != "0"} {
	set_pin_length_extension -hor_extension $::env(FP_IO_HEXTEND) \
		-ver_extension $::env(FP_IO_VEXTEND)
}

if {$::env(FP_IO_VTHICKNESS_MULT) != "" && $::env(FP_IO_HTHICKNESS_MULT) != ""} {
	set_pin_thick_multiplier -hor_multiplier $::env(FP_IO_HTHICKNESS_MULT) \
		-ver_multiplier $::env(FP_IO_VTHICKNESS_MULT)
}

set arg_list [list]
if { $::env(FP_IO_MODE) == 1 } {
	lappend arg_list -random
}

if { $::env(FP_IO_MIN_DISTANCE) != "" } {
	lappend arg_list -min_distance $::env(FP_IO_MIN_DISTANCE)
}

set HMETAL $::env(FP_IO_HLAYER)
set VMETAL $::env(FP_IO_VLAYER)

place_pins {*}$arg_list \
	-random_seed 42 \
	-hor_layers $HMETAL \
	-ver_layers $VMETAL

# create io-ring
make_io_sites -horizontal_site GF_IO_Site -vertical_site GF_IO_Site -corner_site GF_COR_Site -offset 0
place_pad -master gf180mcu_fd_io__dvdd   -row IO_NORTH -location  400.0 pad_dvdd1
place_pad -master gf180mcu_fd_io__in_s   -row IO_NORTH -location  700.0 pad_clk
place_pad -master gf180mcu_fd_io__dvss   -row IO_NORTH -location 1000.0 pad_dvss1

place_pad -master gf180mcu_fd_io__dvdd   -row IO_EAST  -location  400.0 pad_dvdd2
place_pad -master gf180mcu_fd_io__in_s   -row IO_EAST  -location  700.0 pad_din
place_pad -master gf180mcu_fd_io__dvss   -row IO_EAST  -location 1000.0 pad_dvss2

place_pad -master gf180mcu_fd_io__dvdd   -row IO_WEST  -location  400.0 pad_dvdd3
place_pad -master gf180mcu_fd_io__bi_24t -row IO_WEST  -location  700.0 pad_dout
place_pad -master gf180mcu_fd_io__dvss   -row IO_WEST  -location 1000.0 pad_dvss3

place_pad -master gf180mcu_fd_io__dvdd   -row IO_SOUTH -location  400.0 pad_dvdd4
place_pad -master gf180mcu_fd_io__dvss   -row IO_SOUTH -location 1000.0 pad_dvss4

place_corners gf180mcu_fd_io__cor
foreach io_row { IO_EAST IO_WEST IO_SOUTH IO_NORTH } {
    place_io_fill -row $io_row gf180mcu_fd_io__fill10 gf180mcu_fd_io__fill5 gf180mcu_fd_io__fill1
}

connect_by_abutment

write


Overwriting eda/share/openlane/scripts/openroad/ioplacer.tcl


In [ ]:
#@title Run OpenLANE { display-mode: "form" }
!flow.tcl -design .

OpenLane 2023.04.12_0_g23e367a9-conda
All rights reserved. (c) 2020-2022 Efabless Corporation and contributors.
Available under the Apache License, version 2.0. See the LICENSE file for more details.

[INFO]: Using configuration in 'config.tcl'...
[INFO]: PDK Root: /content/eda/share/pdk
[INFO]: Process Design Kit: gf180mcuC
[INFO]: Standard Cell Library: gf180mcu_fd_sc_mcu7t5v0
[INFO]: Optimization Standard Cell Library: gf180mcu_fd_sc_mcu7t5v0
[WARNING]: DIODE_INSERTION_STRATEGY is now deprecated; use GRT_REPAIR_ANTENNAS, DIODE_ON_PORTS and RUN_HEURISTIC_DIODE_INSERTION instead.
[INFO]: DIODE_INSERTION_STRATEGY set to 0. Setting GRT_REPAIR_ANTENNAS to 0
[INFO]: Run Directory: /content/runs/RUN_2023.06.28_14.45.39
[INFO]: Preparing LEF files for the nom corner...
[INFO]: Preparing LEF files for the min corner...
[INFO]: Preparing LEF files for the max corner...
[WARNING]: GPIO_PADS_VERILOG is not set; cannot read as a blackbox
[STEP 1]
[INFO]: Running Synthesis (log: runs/RUN_2023.06.